In [2]:
!pip install gensim
#from predict_demo import model_fn
import sagemaker

     |████████████████████████████████| 24.2 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 74.0 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.1.1-py3-none-any.whl size=112413 sha256=e6f63ce9f258cbe9d06e359a260f3e0148d6b3d4e7b1354c743ea513447f2758
  Stored in directory: /home/ec2-user/.cache/pip/wheels/07/d8/70/da31eb9ec5ff49ac74f1a5944704029f0b6fd74fdf318ec300
Successfully built smart-open
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
import boto3
import pickle
import pandas as pd
from io import StringIO
import io

# !pip install gensim
from gensim.models.word2vec import Word2Vec
from sklearn.cluster import KMeans;
from sklearn.neighbors import KDTree
import numpy as np

In [4]:
from sklearn.datasets import load_iris

Load original dataset

In [4]:
s3 = boto3.client('s3')

object = s3.get_object(Bucket='inroads-test-bucket1',Key='data/Vectorized/inroads_vectorized.csv')
inroads_vectorized = pd.read_csv(io.BytesIO(object['Body'].read()), encoding='utf8', index_col=0)

inroads_vectorized=inroads_vectorized.rename(columns = {'tokens':'token'})

inroads_vectorized.shape

(10061, 302)

In [5]:
s3 = boto3.client('s3')

object = s3.get_object(Bucket='inroads-test-bucket1',Key='data/Vectorized/old_vectorized.csv')
old_vectorized = pd.read_csv(io.BytesIO(object['Body'].read()), encoding='utf8', index_col=0)

old_vectorized.shape

(941, 302)

## Format 

old tokens

In [6]:
old_vocab = list()

for i in range(0,len(old_vectorized)):
    test = old_vectorized.iloc[i].token
    old_vocab.append(test)

old_vocab_df = pd.DataFrame(old_vocab, columns=["token"])

In [7]:
old_vectors_2 = list()

for i in range(0,len(old_vectorized)):
    test = old_vectorized.iloc[i, 2:].values
    old_vectors_2.append(test)

In [8]:
old_array = np.array(old_vectors_2)

In [9]:
old_vectors_df_2 = old_vectorized.drop(['token','vector'], axis=1)
old_vectors_df_2.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.009340,-0.011540,-0.005011,0.170543,-0.070042,-0.022673,0.035479,-0.033519,0.038251,0.045745,...,0.027858,0.024611,-0.013685,0.042488,0.054621,-0.036383,0.018394,-0.045638,0.025203,-0.052491
1,0.073121,-0.047603,0.091147,0.162190,-0.052149,-0.108441,-0.028311,-0.041613,-0.026646,-0.015232,...,0.018856,0.007211,-0.080637,0.157568,-0.035902,-0.001056,-0.014947,-0.114950,-0.121481,0.031619
2,0.013254,0.050222,-0.089445,0.083483,-0.044747,-0.065899,0.019941,-0.052136,0.037055,-0.078439,...,0.043838,0.002926,-0.050873,-0.012946,-0.007152,0.148603,0.109223,-0.019303,0.029999,0.042774
3,0.017232,-0.138262,-0.069550,0.161501,-0.024385,-0.121781,-0.035268,0.007963,0.047652,-0.019447,...,0.027821,0.020316,-0.028523,0.028286,-0.049850,-0.049433,0.133848,0.008867,-0.076711,0.042498
4,-0.056970,0.020302,-0.024677,-0.048865,0.064803,-0.033035,-0.029016,-0.020143,0.038997,0.015680,...,-0.071911,-0.002086,0.023876,0.077053,0.024279,-0.043584,-0.053606,0.007344,0.018627,-0.046889


new text data

In [10]:
## list of just tokens
inroads_vocab = list()

for i in range(0,len(inroads_vectorized)):
    test = inroads_vectorized.iloc[i].token
    inroads_vocab.append(test)

inroads_vocab_df = pd.DataFrame(inroads_vocab, columns=["token"])

In [11]:
## list of arrays of vectors
inroads_vectors_2 = list()

for i in range(0,len(inroads_vectorized)):
    test = inroads_vectorized.iloc[i, 2:].values
    inroads_vectors_2.append(test)

In [12]:
## formatted as numpy array
import numpy as np
inroads_array = np.array(inroads_vectors_2)

In [13]:
inroads_vectorized.head()

,token,vector,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,research,[ 0.18854871 -0.0510548 -0.05103487 0.012128...,0.188549,-0.051055,-0.051035,0.012129,0.018650,-0.108756,-0.054335,-0.003939,...,0.067170,0.014436,0.027066,-0.011628,-0.076336,0.020410,0.032563,-0.045787,-0.077532,-0.084312
1,article,[-0.08915062 -0.03244422 0.0660668 -0.083048...,-0.089151,-0.032444,0.066067,-0.083048,-0.025339,-0.115493,-0.082126,-0.015637,...,-0.031248,0.035671,0.022658,0.047292,-0.071116,-0.075786,0.015880,0.021262,-0.051069,-0.082306
2,personality,[ 0.08098853 0.07145718 -0.03669713 0.051603...,0.080989,0.071457,-0.036697,0.051603,-0.098144,-0.004233,0.047631,0.025854,...,-0.020227,0.035703,-0.040952,-0.026335,0.001954,0.001704,0.098474,-0.053931,-0.007621,0.002607
3,social,[ 4.81807813e-02 -2.48146318e-02 -4.25761268e-...,0.048181,-0.024815,-0.042576,0.044887,-0.072843,-0.014483,-0.092501,-0.036309,...,0.001294,-0.023781,-0.000153,-0.002167,0.062039,0.115820,0.100294,0.055571,0.046550,0.041450
4,psychology,[-2.08396418e-03 8.44885111e-02 1.47056533e-...,-0.002084,0.084489,0.014706,0.073587,-0.098434,-0.009841,-0.012736,-0.084855,...,0.072306,-0.008926,-0.072427,-0.022924,-0.045194,-0.044142,0.019417,0.026820,0.013483,-0.025351


In [14]:
## vectors only as df
inroads_vectors_df_2 = inroads_vectorized.drop(['token','vector'], axis=1)
inroads_vectors_df_2.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.188549,-0.051055,-0.051035,0.012129,0.018650,-0.108756,-0.054335,-0.003939,0.021723,-0.048769,...,0.067170,0.014436,0.027066,-0.011628,-0.076336,0.020410,0.032563,-0.045787,-0.077532,-0.084312
1,-0.089151,-0.032444,0.066067,-0.083048,-0.025339,-0.115493,-0.082126,-0.015637,0.010993,-0.020402,...,-0.031248,0.035671,0.022658,0.047292,-0.071116,-0.075786,0.015880,0.021262,-0.051069,-0.082306
2,0.080989,0.071457,-0.036697,0.051603,-0.098144,-0.004233,0.047631,0.025854,-0.024231,-0.023942,...,-0.020227,0.035703,-0.040952,-0.026335,0.001954,0.001704,0.098474,-0.053931,-0.007621,0.002607
3,0.048181,-0.024815,-0.042576,0.044887,-0.072843,-0.014483,-0.092501,-0.036309,-0.056641,-0.054676,...,0.001294,-0.023781,-0.000153,-0.002167,0.062039,0.115820,0.100294,0.055571,0.046550,0.041450
4,-0.002084,0.084489,0.014706,0.073587,-0.098434,-0.009841,-0.012736,-0.084855,0.008198,-0.003929,...,0.072306,-0.008926,-0.072427,-0.022924,-0.045194,-0.044142,0.019417,0.026820,0.013483,-0.025351


## Dumping and loading a model

In [14]:
from sklearn.cluster import KMeans
from sagemaker.sklearn.estimator import SKLearn


In [13]:
#kmeans = KMeans(9, random_state=12345)
#kmeans.fit(old_array)

KMeans(n_clusters=9, random_state=12345)

Using the popular python library `joblib`, we can store and load our sklearn based models conveniently and quickly. 

In [107]:
#from joblib import dump, load
#dump(kmeans, 'inroads_model/kmeans_inroads.joblib') 

['inroads_model/kmeans_inroads.joblib']

In [19]:
# for loading the model
#clusterer = load('inroads_model/kmeans_inroads.joblib')

In [20]:
#kmeans.cluster_centers_

array([[-0.00911223, -0.01475857,  0.02156253, ..., -0.01086714,
        -0.02190037,  0.00932974],
       [-0.01985547,  0.00391338,  0.01887953, ...,  0.01553798,
        -0.00253671,  0.01605074],
       [-0.00991414, -0.03176413, -0.01107653, ..., -0.03559942,
        -0.0064654 ,  0.00151505],
       ...,
       [-0.01213695,  0.02188849, -0.0053943 , ..., -0.01972915,
        -0.01170868,  0.01229602],
       [ 0.03164612,  0.00056793,  0.00555031, ...,  0.00383961,
         0.01936551,  0.00123629],
       [-0.02034095, -0.00437334, -0.02058369, ..., -0.01609   ,
        -0.03461457,  0.02567087]])

In [21]:
#clusterer.cluster_centers_

array([[-0.00911223, -0.01475857,  0.02156253, ..., -0.01086714,
        -0.02190037,  0.00932974],
       [-0.01985547,  0.00391338,  0.01887953, ...,  0.01553798,
        -0.00253671,  0.01605074],
       [-0.00991414, -0.03176413, -0.01107653, ..., -0.03559942,
        -0.0064654 ,  0.00151505],
       ...,
       [-0.01213695,  0.02188849, -0.0053943 , ..., -0.01972915,
        -0.01170868,  0.01229602],
       [ 0.03164612,  0.00056793,  0.00555031, ...,  0.00383961,
         0.01936551,  0.00123629],
       [-0.02034095, -0.00437334, -0.02058369, ..., -0.01609   ,
        -0.03461457,  0.02567087]])

## Saving to s3

In [5]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [6]:
bucket_name = bucket
print(bucket_name)

sagemaker-us-east-1-068949824886


In [16]:
## not sure if this actually works so keeping it out for now
#!tar -czf model.tar.gz demo_model/kmeans_demo.joblib
#!aws s3 cp model.tar.gz s3://sagemaker-us-east-1-068949824886/model/model.tar.gz

In [17]:
#key = 'demo_model/kmeans_demo.joblib'

In [18]:
#s3 = boto3.resource('s3')
#s3.meta.client.upload_file(key, bucket_name, 'model.joblib')

One thing I always do is go to my AWS console and double-check that it really is there and it looks as you expect it to look. This has the double effect of letting me know I did something right and reinforcing my confidence in my AWS skill. 

For the sake of coherency and good practice, let's upload the data there too. 

In [192]:
old_vectors_df_2.to_csv('inroads_model/inroads_data_test.csv', columns = old_vectors_df_2.columns)

In [7]:
# should be the name of directory you created to save your features data
data_dir = 'inroads_model/inroads_data_test.csv'

# set prefix, a descriptive name for a directory  
prefix = 'data'

# upload all data to S3
s3_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket_name, key_prefix=prefix)

In [8]:
bucket

'sagemaker-us-east-1-068949824886'

In [136]:
# also, here's code to use if you want to programmatically check that your data has been uploaded successfully
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

data/inroads_data_test.csv
data/iris_data_test.csv
data/sagemaker-scikit-learn-2020-08-26-13-55-58-101/debug-output/training_job_end.ts
data/sagemaker-scikit-learn-2020-08-26-13-55-58-101/output/model.tar.gz
data/sagemaker-scikit-learn-2020-08-26-14-52-15-495/debug-output/training_job_end.ts
data/sagemaker-scikit-learn-2020-08-26-14-52-15-495/output/model.tar.gz
data/sagemaker-scikit-learn-2020-08-26-14-52-24-918/debug-output/training_job_end.ts
data/sagemaker-scikit-learn-2020-08-26-14-52-24-918/output/model.tar.gz
data/sagemaker-scikit-learn-2020-08-26-15-00-25-570/debug-output/training_job_end.ts
data/sagemaker-scikit-learn-2020-08-26-15-00-25-570/output/model.tar.gz
data/sagemaker-scikit-learn-2020-08-26-16-56-31-937/debug-output/training_job_end.ts
data/sagemaker-scikit-learn-2020-08-26-16-56-31-937/output/model.tar.gz
data/sagemaker-scikit-learn-2020-08-26-17-55-52-979/debug-output/training_job_end.ts
data/sagemaker-scikit-learn-2020-08-26-17-55-52-979/output/model.tar.gz
data/sa

Another important thing is that the model will be saved via joblib (we specified this at the end of the `__main__` function in the training script. 

## DEPLOYING an already trained model

You can use an already pretrained model in AWS using SKLearn**Model** rather than just SKLearn. 

I'd previously saved the model artefacts as a joblib file, however the default files expected tend to be tar.gz files so AWS will have no problem recognising that kind of file. 

Two cells down I've provided the URL of my model's parameters and then I've instantiated the enpoint. 
When doing so, I am effectively telling AWS to get the artefacts from S3 and the instructions on how to use them from my `predict_demo.py` file (located in the source_dir). 

Now when this endpoint is deployed, it will always know to follow the instructions in the predict file, even when accessed via Lambda function. 

In [12]:
from sagemaker.sklearn import SKLearnModel

In [25]:
'''# the predictor endpoint expects something stored in an S3 bucket
# here I've just copied the url to my model artefacts from the AWS console
model_key = 'model/model.tar.gz' 
data_location = 's3://{}/{}'.format(bucket_name, model_key) 

model = SKLearnModel(model_data=model_url, # pointing to the model artefacts - our learned weights and coeffs
                     role = role, # using the specified ARN
                     framework_version='0.4.0', 
                     entry_point='predict_demo.py', # which file to go to to find the respective functions
                     source_dir='demo_model/train') # directory to access
#                      predictor_cls=StringPredictor) '''

"# the predictor endpoint expects something stored in an S3 bucket\n# here I've just copied the url to my model artefacts from the AWS console\nmodel_key = 'model/model.tar.gz' \ndata_location = 's3://{}/{}'.format(bucket_name, model_key) \n\nmodel = SKLearnModel(model_data=model_url, # pointing to the model artefacts - our learned weights and coeffs\n                     role = role, # using the specified ARN\n                     framework_version='0.4.0', \n                     entry_point='predict_demo.py', # which file to go to to find the respective functions\n                     source_dir='demo_model/train') # directory to access\n#                      predictor_cls=StringPredictor) "

In [26]:
#predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [93]:
# when using deploy AWS will look to find our model using `model_fn`; the entry point is the same as it was for when we trained 
# the estimator
from sagemaker.predictor import RealTimePredictor
#Uncomment the 3 lines below when we'll be using a string based predictor. Atm this kmeans model is just being used with the iris set

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

# OPTIONAL:  Training and saving an AWS style estimator
Although not necessary for this project, it's still handy to know, especially if you have to train a model on a large amount of data or require GPUs. 

For deployment purposes we need the AWS SKLearn Estimator object. This acts as a dockerised container that allows AWS to interact with our sklearn model. I'll be training the AWS model below. I've written a training script in demo_model/train/train.py. 

In [9]:
train_channel = sagemaker.session.s3_input(s3_data, content_type='text/csv')
data_channels = {'train': train_channel}

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [15]:
%%time
# Train your estimator on S3 training data

output_path = 's3://{}/{}'.format(bucket_name, prefix)

estimator = SKLearn(entry_point='train.py',     # name of the training script AWS should access
                    source_dir = 'inroads_model/train', # dir with training script
                    role=role,   # the role we stated higher up in the nb
                    train_instance_count=1,
                    framework_version="0.23-1",  # no need to change this
                    train_instance_type='ml.m4.xlarge' ,  # no need to change this, unless you're using pytorch models OR want to include GP
                    output_path = output_path,  # no need to change this, unless you want a different output location for the file
                    sagemaker_session = sagemaker_session
#                    predictor_cls = 
#                     hyperparameters = {
#                                        'n_clusters':4,
#                                         }
                   )

CPU times: user 11.5 ms, sys: 0 ns, total: 11.5 ms
Wall time: 10.6 ms


In [17]:
estimator.fit(data_channels)

2020-09-07 16:18:50 Starting - Starting the training job...
2020-09-07 16:18:54 Starting - Launching requested ML instances......
2020-09-07 16:20:07 Starting - Preparing the instances for training.........
2020-09-07 16:21:46 Downloading - Downloading input data
2020-09-07 16:21:46 Training - Downloading the training image......
2020-09-07 16:22:49 Training - Training image download completed. Training in progress..2020-09-07 16:22:49,801 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2020-09-07 16:22:49,803 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-09-07 16:22:49,815 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-09-07 16:22:53,238 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-09-07 16:22:53,252 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-09-07 16:22:53,267 sagemaker-training

In [18]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', )

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------------!

### Real Time Prediction

In [19]:
len(inroads_vocab)

NameError: name 'inroads_vocab' is not defined

In [25]:
data = {"terms":
         [ ' feel ',
         ' love ', 
 ' good ', 
 ' comfort ', 
 ' hope ', 
 ' happy ', 
 ' wish ', 
 ' sorrow ', 
 ' pity ', 
 ' sadness ', 
 ' person ', 
 ' grief ', 
 ' live ', 
 ' people ', 
 ' kindness ', 
 ' compassion ', 
 ' regret ', 
 ' pain ', 
 ' bless ', 
 ' jesus ', 
 ' mifepristone ', 
 ' misoprostol ', 
 ' surgical ', 
 ' dilatation ', 
 ' pregnancy ', 
 ' complications ', 
 ' ultrasound ', 
 ' uterine ', 
 ' uterus ', 
 ' cervical ', 
 ' antenatal ', 
 ' surgery ', 
 ' medical ', 
 ' trimester ', 
 ' doctor ', 
 ' antibiotics ', 
 ' regimen ', 
 ' referral ', 
 ' hospital ', 
 ' postabortion ', 
 ' sidewalk ', 
 ' yellow ', 
 ' feet ', 
 ' legs ', 
 ' sleep ', 
 ' bleed ', 
 ' toilet ', 
 ' bicycle ', 
 ' purple ', 
 ' bike ', 
 ' garbage ', 
 ' stomach ', 
 ' stick ', 
 ' abdomen ', 
 ' room ', 
 ' boat ', 
 ' night ', 
 ' shop ', 
 ' scissor ', 
 ' shoot ', 
 ' government ', 
 ' movement ', 
 ' coalition ', 
 ' organizations ', 
 ' right ', 
 ' organization ', 
 ' policy ', 
 ' opposition ', 
 ' national ', 
 ' campaign ', 
 ' support ', 
 ' action ', 
 ' congress ', 
 ' advocacy ', 
 ' state ', 
 ' political ', 
 ' mobilization ', 
 ' legislation ', 
 ' power ', 
 ' equality ', 
 ' health ', 
 ' risk ', 
 ' access ', 
 ' care ', 
 ' childcare ', 
 ' education ', 
 ' quality ', 
 ' fertility ', 
 ' children ', 
 ' women ', 
 ' availability ', 
 ' cost ', 
 ' social ', 
 ' contraceptive ', 
 ' condition ', 
 ' safety ', 
 ' pregnancy ', 
 ' individual ', 
 ' reproduction ', 
 ' healthcare ', 
 ' religion ', 
 ' concept ', 
 ' value ', 
 ' moral ', 
 ' morality ', 
 ' religious ', 
 ' argument ', 
 ' political ', 
 ' ethics ', 
 ' norms ', 
 ' christianity ', 
 ' process ', 
 ' conflict ', 
 ' belief ', 
 ' society ', 
 ' identity ', 
 ' autonomy ', 
 ' social ', 
 ' consensus ', 
 ' conception ', 
 ' violence ', 
 ' injustice ', 
 ' harm ', 
 ' shame ', 
 ' fear ', 
 ' rape ', 
 ' abuse ', 
 ' hatred ', 
 ' prejudice ', 
 ' stigmatization ', 
 ' stigma ', 
 ' criminalization ', 
 ' cruelty ', 
 ' crime ', 
 ' promiscuity ', 
 ' anxiety ', 
 ' danger ', 
 ' discrimination ', 
 ' threat ', 
 ' suffer ', 
 ' wife ', 
 ' brother ', 
 ' cousin ', 
 ' husband ', 
 ' father ', 
 ' daughter ', 
 ' aunt ', 
 ' sister ', 
 ' boyfriend ', 
 ' grandmother ', 
 ' friend ', 
 ' grandfather ', 
 ' uncle ', 
 ' girl ', 
 ' funeral ', 
 ' niece ', 
 ' nephew ', 
 ' girlfriend ', 
 ' auntie ', 
 ' soldier ', 
 ' june ', 
 ' october ', 
 ' december ', 
 ' april ', 
 ' august ', 
 ' summer ', 
 ' july ', 
 ' february ', 
 ' january ', 
 ' morning ', 
 ' week ', 
 ' afternoon ', 
 ' march ', 
 ' november ', 
 ' september ', 
 ' thursday ', 
 ' night ', 
 ' year ', 
 ' saturday ', 
 ' month ']}


In [31]:
runtime = boto3.client('sagemaker-runtime')

inp = data['terms']
#old_vocab

#inp = inroads_vocab[10001:]
payload = bytes(str(data['terms']), 'utf8')

# csv serialization
response = runtime.invoke_endpoint(
    EndpointName='sagemaker-scikit-learn-2020-09-06-22-48-07-194',
    Body=payload,
    ContentType='text/csv')

result = list(response['Body'].read().decode().strip("[]").split(" "))


In [32]:
output = dict()

In [33]:
for i in range(len(result)):
    category = result[i].strip()
    test_term =inp[i]
    output[test_term]=category

In [34]:
output

{' feel ': '8',
 ' love ': '8',
 ' good ': '8',
 ' comfort ': '8',
 ' hope ': '8',
 ' happy ': '8',
 ' wish ': '8',
 ' sorrow ': '8',
 ' pity ': '8',
 ' sadness ': '8',
 ' person ': '8',
 ' grief ': '8',
 ' live ': '8',
 ' people ': '8',
 ' kindness ': '8',
 ' compassion ': '8',
 ' regret ': '8',
 ' pain ': '8',
 ' bless ': '8',
 ' jesus ': '8',
 ' mifepristone ': '8',
 ' misoprostol ': '8',
 ' surgical ': '8',
 ' dilatation ': '8',
 ' pregnancy ': '8',
 ' complications ': '8',
 ' ultrasound ': '8',
 ' uterine ': '8',
 ' uterus ': '8',
 ' cervical ': '8',
 ' antenatal ': '8',
 ' surgery ': '8',
 ' medical ': '8',
 ' trimester ': '8',
 ' doctor ': '8',
 ' antibiotics ': '8',
 ' regimen ': '8',
 ' referral ': '8',
 ' hospital ': '8',
 ' postabortion ': '8',
 ' sidewalk ': '8',
 ' yellow ': '8',
 ' feet ': '8',
 ' legs ': '8',
 ' sleep ': '8',
 ' bleed ': '8',
 ' toilet ': '8',
 ' bicycle ': '8',
 ' purple ': '8',
 ' bike ': '8',
 ' garbage ': '8',
 ' stomach ': '8',
 ' stick ': '8',
 ' a

In [124]:
output_df = pd.DataFrame(list(output.items()),columns = ['token','cluster']) 
output_df.groupby(['cluster']).agg(['count'])

,token
,count
cluster,
0,1955
1,735
2,898
3,574
4,568
5,444
6,1915
7,899


In [132]:
output_df.pivot(columns="cluster", values="token").sort_values(by='3').to_csv('clusters.csv',index=False)

## Testing Trained Model

In [88]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
from predict_demo2 import input_fn, predict_fn
from utils import lookup_table,get_lookup_table, format_input, upload_lookup_table

In [62]:
inp = ["aborton"]
payload = bytes(str(inp), 'utf8')

In [67]:
formatted_input_data = format_input(inp)

In [70]:
vector_table = get_lookup_table()

In [71]:
vector_table.head()

,token,vector,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,abstract,[ 9.34010372e-03 -1.15401922e-02 -5.01083815e-...,0.009340,-0.011540,-0.005011,0.170543,-0.070042,-0.022673,0.035479,-0.033519,...,0.027858,0.024611,-0.013685,0.042488,0.054621,-0.036383,0.018394,-0.045638,0.025203,-0.052491
1,absence,[ 0.07312082 -0.04760321 0.09114736 0.162189...,0.073121,-0.047603,0.091147,0.162190,-0.052149,-0.108441,-0.028311,-0.041613,...,0.018856,0.007211,-0.080637,0.157568,-0.035902,-0.001056,-0.014947,-0.114950,-0.121481,0.031619
2,abundance,[ 1.32538220e-02 5.02222404e-02 -8.94451290e-...,0.013254,0.050222,-0.089445,0.083483,-0.044747,-0.065899,0.019941,-0.052136,...,0.043838,0.002926,-0.050873,-0.012946,-0.007152,0.148603,0.109223,-0.019303,0.029999,0.042774
3,reality,[ 0.01723179 -0.13826157 -0.06955002 0.161501...,0.017232,-0.138262,-0.069550,0.161501,-0.024385,-0.121781,-0.035268,0.007963,...,0.027821,0.020316,-0.028523,0.028286,-0.049850,-0.049433,0.133848,0.008867,-0.076711,0.042498
4,beautiful,[-5.69702238e-02 2.03019492e-02 -2.46770643e-...,-0.056970,0.020302,-0.024677,-0.048865,0.064803,-0.033035,-0.029016,-0.020143,...,-0.071911,-0.002086,0.023876,0.077053,0.024279,-0.043584,-0.053606,0.007344,0.018627,-0.046889


In [90]:
vectorised_input,vector_table_new = lookup_table(vector_table, formatted_input_data)

AttributeError: 'numpy.ndarray' object has no attribute 'array_split'

In [33]:
upload_lookup_table()

{'ResponseMetadata': {'RequestId': '8DE5606AAC4E1276',
  'HostId': 'D4am2q07OFr4T091/nizMgKkwNHSr4lBbe2gjXuHBJh0OmNKL8Zm7rr303ZAxliDH7bMXNM8vmc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'D4am2q07OFr4T091/nizMgKkwNHSr4lBbe2gjXuHBJh0OmNKL8Zm7rr303ZAxliDH7bMXNM8vmc=',
   'x-amz-request-id': '8DE5606AAC4E1276',
   'date': 'Thu, 03 Sep 2020 18:30:53 GMT',
   'etag': '"46fcaa0492dece6cc4969ff381ec54f0"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"46fcaa0492dece6cc4969ff381ec54f0"'}

In [134]:
predict_fn(inp, predictor)

Determining nearest cluster.


array([8, 3, 5, 5, 7], dtype=int32)

# END THE SESSION; DELETE THE BUCKET AND ENDPOINT
Uncomment the cell below

In [33]:
#sagemainroads_vectors_df_2endpoint()
# bucket_to_delete = boto3.resource('s3').Bucket(bucket)
# bucket_to_delete.objects.all().delete()

TypeError: delete_endpoint() missing 1 required positional argument: 'endpoint_name'

In [20]:
predictor.endpoint

'sagemaker-scikit-learn-2020-09-07-16-18-49-857'

In [198]:
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++');
    idx = kmeans_clustering.fit_predict(word_vectors);
    
    return kmeans_clustering, kmeans_clustering.cluster_centers_, idx;

In [199]:
kmodel,centers,clusters = clustering_on_wordvecs(old_vectors_2, 9);
centroid_map = dict(zip(old_vocab, clusters));

In [200]:
centroid_map

{'abstract': 8,
 'absence': 8,
 'abundance': 2,
 'reality': 8,
 'beautiful': 4,
 'complexity': 8,
 'creation': 2,
 'finality': 8,
 'truth': 8,
 'burial': 7,
 'life': 8,
 'complicate': 8,
 'natural': 8,
 'peace': 2,
 'universal': 8,
 'work': 2,
 'nuance': 8,
 'complex': 8,
 'vision': 8,
 'security': 5,
 'save': 1,
 'live': 4,
 'stable': 8,
 'option': 5,
 'limbo': 1,
 'circle': 7,
 'square': 7,
 'triangle': 7,
 'rectangle': 7,
 'sphere': 0,
 'blue': 7,
 'color': 2,
 'orange': 7,
 'yellow': 7,
 'green': 7,
 'purple': 7,
 'black': 2,
 'white': 7,
 'darkness': 1,
 'light': 0,
 'radiance': 1,
 'actor': 4,
 'entertainment': 2,
 'music': 0,
 'literature': 0,
 'performance': 2,
 'theater': 7,
 'drama': 0,
 'poetry': 0,
 'artist': 4,
 'band': 7,
 'musical': 0,
 'book': 0,
 'film': 0,
 'documentary': 0,
 'misinformation': 1,
 'advertisement': 6,
 'radio': 7,
 'silence': 1,
 'media': 2,
 'journalism': 0,
 'social': 2,
 'what': 8,
 'billboard': 7,
 'poster': 7,
 'flyer': 7,
 'talk': 1,
 'show': 2,


In [63]:
output

{' feel ': '4',
 ' love ': '4',
 ' good ': '4',
 ' comfort ': '4',
 ' hope ': '4',
 ' happy ': '4',
 ' wish ': '4',
 ' sorrow ': '4',
 ' pity ': '4',
 ' sadness ': '4',
 ' person ': '4',
 ' grief ': '4',
 ' live ': '4',
 ' people ': '4',
 ' kindness ': '4',
 ' compassion ': '4',
 ' regret ': '4',
 ' pain ': '4',
 ' bless ': '4',
 ' jesus ': '4',
 ' mifepristone ': '4',
 ' misoprostol ': '4',
 ' surgical ': '4',
 ' dilatation ': '4',
 ' pregnancy ': '4',
 ' complications ': '4',
 ' ultrasound ': '4',
 ' uterine ': '4',
 ' uterus ': '4',
 ' cervical ': '4',
 ' antenatal ': '4',
 ' surgery ': '4',
 ' medical ': '4',
 ' trimester ': '4',
 ' doctor ': '4',
 ' antibiotics ': '4',
 ' regimen ': '4',
 ' referral ': '4',
 ' hospital ': '4',
 ' postabortion ': '4',
 ' sidewalk ': '4',
 ' yellow ': '4',
 ' feet ': '4',
 ' legs ': '4',
 ' sleep ': '4',
 ' bleed ': '4',
 ' toilet ': '4',
 ' bicycle ': '4',
 ' purple ': '4',
 ' bike ': '4',
 ' garbage ': '4',
 ' stomach ': '4',
 ' stick ': '4',
 ' a